<div align="center">

# Final Exam

## Programming, Algorithms and Data Structures [KAN-CDSCO2402U]

**Beata Derouin (152415)**



*MSc in Business Administration and Data Science*

</div>


## Dependencies
##### Run the following code block to make sure all dependencies are installed

In [1]:
%pip install flask folium pandas ipython

Note: you may need to restart the kernel to use updated packages.


## Imports

In [3]:
from flask import Flask, request, jsonify
import random
import folium # Folium: Chosen for its ability to create interactive maps easily.
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display

## Flask App Initialization

In [5]:
app = Flask(__name__) # Flask: Used for its lightweight and efficient handling of RESTful APIs.

## Global Functions

In [7]:
def generate_random_price():
    """Generates a random price between 50 and 300 DKK for classes."""
    return round(random.uniform(50, 300), 2)

def generate_random_date():
    """Generates a random date within the next 7 days."""
    start_date = datetime.now()
    random_days = random.randint(0, 6)
    random_time = random.randint(6, 20)
    return (start_date + timedelta(days=random_days)).strftime(f"%Y-%m-%d {random_time}:00")

def generate_class_dataframe(classes):
    """Creates a Pandas DataFrame from the Pilates classes."""
    return pd.DataFrame({
        'Class_ID': [cls.class_id for cls in classes],
        'Name': [cls.name for cls in classes],
        'Latitude': [cls.latitude for cls in classes],
        'Longitude': [cls.longitude for cls in classes],
        'Price': [cls.price for cls in classes],
        'Date': [cls.date for cls in classes],
        'Area': [cls.area for cls in classes],
        'Capacity': [cls.capacity for cls in classes]
    })

## Classes

### PilatesClass

In [9]:
class PilatesClass:
    id_counter = 1 # ensures unique class IDs

    def __init__(self, name, date, duration, area, address, instructor, difficulty, price, latitude, longitude):
        self.class_id = PilatesClass.id_counter
        PilatesClass.id_counter += 1

        self.name = name
        self.date = date
        self.duration = duration
        self.area = area
        self.address = address
        self.instructor = instructor
        self.difficulty = difficulty
        self.price = price
        self.latitude = latitude
        self.longitude = longitude
        self.capacity = 10  # Default capacity for classes
        self.participants = set()  # Track participants by user ID

    def book_spot(self, user):
        """Books a spot for the user if there is availability."""
        if user.user_id in self.participants:
            return False, f"User {user.name} has already booked this class."
        if self.capacity > 0:
            self.participants.add(user.user_id)
            self.capacity -= 1
            return True, f"User {user.name} successfully booked class {self.name}. Remaining capacity: {self.capacity}."
        return False, f"Class {self.name} is fully booked."
    
    def show_details(self):
        """Displays the details of the Pilates class."""
        return (f"Class ID: {self.class_id}, Name: {self.name}, Date: {self.date}, Duration: {self.duration}, "
                f"Area: {self.area}, Address: {self.address}, Instructor: {self.instructor}, "
                f"Difficulty Level: {self.difficulty}, Price: {self.price:.2f} DKK, Remaining Capacity: {self.capacity}")

#### Subclasses of PilatesClass

In [11]:
class BeginnerPilatesClass(PilatesClass):
    def __init__(self, name, date, duration, area, address, instructor, difficulty, price, latitude, longitude):
        super().__init__(name, date, duration, area, address, instructor, difficulty, price, latitude, longitude)
        self.type = "Beginner"

    def book_spot(self, user):
        """Overrides the booking logic for beginner classes."""
        if self.capacity == 0:
            return False, f"Beginner Class {self.name} is full."
        return super().book_spot(user)

class IntermediatePilatesClass(PilatesClass):
    def __init__(self, name, date, duration, area, address, instructor, difficulty, price, latitude, longitude):
        super().__init__(name, date, duration, area, address, instructor, difficulty, price, latitude, longitude)
        self.type = "Intermediate"

    def book_spot(self, user):
        """Overrides the booking logic for intermediate classes."""
        if self.capacity == 0:
            return False, f"Intermediate Class {self.name} is full."
        return super().book_spot(user)

class AdvancedPilatesClass(PilatesClass):
    def __init__(self, name, date, duration, area, address, instructor, difficulty, price, latitude, longitude):
        super().__init__(name, date, duration, area, address, instructor, difficulty, price, latitude, longitude)
        self.type = "Advanced"

    def book_spot(self, user):
        """Overrides the booking logic for advanced classes."""
        if self.capacity == 0:
            return False, f"Advanced Class {self.name} is full."
        return super().book_spot(user)

### User

In [13]:
class User:
    id_counter = 1

    def __init__(self, name, email):
        self.user_id = User.id_counter
        User.id_counter += 1
        self.name = name
        self.email = email.lower()  # Store email in lowercase for easier checking
        self.is_student = "student" in self.email  # Automatically check if user is a student

    def get_discounted_price(self, price):
        """Calculates the discounted price for students (10% off)."""
        return price * 0.90 if self.is_student else price


### PilatesPlatform

In [15]:
class PilatesPlatform:
    def __init__(self):
        self.classes = []
        self.users = []

    def add_class(self, pilates_class):
        """Adds a Pilates class to the platform."""
        self.classes.append(pilates_class)

    def register_user(self, name, email):
        """Registers a new user with email."""
        user = User(name, email)
        self.users.append(user)
        discount_msg = "You qualify for a 10% student discount!" if user.is_student else ""
        print(f"\nWelcome {name}! {discount_msg}")
        return user

    def filter_classes_by_area(self, area):
        """Filters classes based on the area."""
        return [cls for cls in self.classes if cls.area.lower() == area.lower()]

    def quicksort_classes(self, classes):
        """Implements QuickSort to sort classes by price."""
        if len(classes) <= 1:
            return classes
        pivot = classes[0]
        less = [cls for cls in classes[1:] if cls.price <= pivot.price]
        greater = [cls for cls in classes[1:] if cls.price > pivot.price]
        return self.quicksort_classes(less) + [pivot] + self.quicksort_classes(greater)

    def display_class_details(cls):
        """Polymorphic method to display class details."""
        if isinstance(cls, BeginnerPilatesClass):
            return f"Beginner Class: {cls.show_details()}"
        elif isinstance(cls, IntermediatePilatesClass):
            return f"Intermediate Class: {cls.show_details()}"
        elif isinstance(cls, AdvancedPilatesClass):
            return f"Advanced Class: {cls.show_details()}"

    def book_class(self, user, class_id):
        """Books a class for the user with enhanced error handling."""
        try:
            # Validate class_id
            if not isinstance(class_id, int):
                raise ValueError("Class ID must be a number")
            
            # Find the class
            target_class = next((cls for cls in self.classes if cls.class_id == class_id), None)
            if not target_class:
                raise ValueError(f"Class with ID {class_id} not found")
            
            # Attempt booking
            success, message = target_class.book_spot(user)
            if success:
                final_price = user.get_discounted_price(target_class.price)
                discount_info = "(10% student discount applied)" if user.is_student else ""
                print(f"\n=== Booking Confirmation ===")
                print(f"✓ {user.name}, your booking is confirmed!")
                print(f"Class: {target_class.name}")
                print(f"Date: {target_class.date}")
                print(f"Price: {final_price:.2f} DKK {discount_info}")
                print(f"Remaining Capacity: {target_class.capacity}")
                print("===========================")
            return success
            
        except ValueError as e:
            print(f"\nBooking Error: {str(e)}")
            return False
        except Exception as e:
            print(f"\nUnexpected error during booking: {str(e)}")
            return False


## Flask route for booking

In [17]:
@app.route('/book-class', methods=['GET'])
def book_class_api():
    """API endpoint to handle booking requests."""
    class_id = int(request.args.get('class_id'))
    user_id = int(request.args.get('user_id'))

    user = next((u for u in platform.users if u.user_id == user_id), None)
    if not user:
        return jsonify({"success": False, "message": "User not found."})

    success = platform.book_class(user, class_id)
    if success:
        return jsonify({"success": True, "message": f"Class ID {class_id} booked successfully!"})
    else:
        return jsonify({"success": False, "message": f"Booking failed. Class ID {class_id} might be full or already booked by the user."})


## Function to ensure interactive booking from the map

In [19]:
def display_map_with_booking(platform, user, sample_fraction=0.1):
    """Displays a Folium map with interactive booking options for Pilates classes."""
    df_classes = generate_class_dataframe(platform.classes)
    sampled_classes = df_classes.sample(frac=sample_fraction, random_state=1)

    map_copenhagen = folium.Map(location=[55.6761, 12.5683], zoom_start=12)

    for _, cls in sampled_classes.iterrows():
        final_price = user.get_discounted_price(cls['Price'])
        discount_info = "(10% student discount applied)" if user.is_student else ""
        iframe = folium.IFrame(
            f"""
            <div id="class-{cls['Class_ID']}">
                <h4>{cls['Name']}</h4>
                <p><strong>Price:</strong> {final_price:.2f} DKK {discount_info}</p>
                <p><strong>Date:</strong> {cls['Date']}</p>
                <p id="capacity-{cls['Class_ID']}"><strong>Remaining Capacity:</strong> {cls['Capacity']}</p>
                <div id="booking-section-{cls['Class_ID']}">
                    <button onclick='bookClass({cls['Class_ID']})'>Book Class</button>
                </div>
            </div>
            <script>
                function bookClass(classId) {{
                    // Get current capacity
                    const capacityElement = document.getElementById(`capacity-${{classId}}`);
                    const currentCapacity = parseInt(capacityElement.textContent.split(': ')[1]);
                    
                    if (currentCapacity > 0) {{
                        // Update capacity
                        const newCapacity = currentCapacity - 1;
                        capacityElement.innerHTML = `<strong>Remaining Capacity:</strong> ${{newCapacity}}`;
                        
                        // Replace booking section with confirmation message
                        const bookingSection = document.getElementById(`booking-section-${{classId}}`);
                        bookingSection.innerHTML = `
                            <div style="color: green; margin-top: 10px;">
                                <strong>✓ Booking Confirmed!</strong><br>
                                Class successfully booked for {user.name}
                            </div>
                        `;

                        // Update the actual platform data
                        for (let cls of platform.classes) {{
                            if (cls.class_id === classId) {{
                                cls.capacity = newCapacity;
                                break;
                            }}
                        }}
                    }} else {{
                        alert('Sorry, this class is fully booked!');
                    }}
                }}
            </script>
            """,
            width=300,
            height=200
        )
        popup = folium.Popup(iframe, max_width=300)
        folium.Marker(
            location=(cls['Latitude'], cls['Longitude']),
            popup=popup,
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(map_copenhagen)

    return map_copenhagen

## Sample Data

In [21]:
def initialize_sample_data(platform):
    """Creates sample usage with Pilates classes and users."""
    areas = ["København Ø", "København K", "København N", "København V", "København S", "København NV"]
    instructors = ["Sarah", "Marie", "Philip", "Magnus"]
    difficulties = ["Beginner", "Intermediate", "Advanced"]
    coordinates = {
        "København Ø": (55.704, 12.579),
        "København K": (55.675, 12.568),
        "København N": (55.692, 12.543),
        "København V": (55.667, 12.541),
        "København S": (55.652, 12.590),
        "København NV": (55.700, 12.530),
    }

    # Add sample classes
    for _ in range(50):
        area = random.choice(areas)
        latitude, longitude = coordinates[area]
        platform.add_class(PilatesClass(
            name=f"Pilates Session {_+1}",
            date=generate_random_date(),
            duration=f"{random.randint(1, 2)}h",
            area=area,
            address=f"Studio {random.randint(1, 10)}, Copenhagen",
            instructor=random.choice(instructors),
            difficulty=random.choice(difficulties),
            price=generate_random_price(),
            latitude=latitude,
            longitude=longitude,
        ))

## Testing

In [43]:
def test_pilates_platform():
    # Create platform instance
    platform = PilatesPlatform()

    # Add a test class
    test_class = PilatesClass(
        name="Test Pilates Class",
        date="2024-12-20 10:00",
        duration="1h",
        area="København Ø",
        address="Test Studio 1",
        instructor="Sarah",
        difficulty="Beginner",
        price=150.00,
        latitude=55.704,
        longitude=12.579
    )
    platform.add_class(test_class)
    
    # Register a test user
    user = platform.register_user("Test User", "test.student@university.dk")
    
    # Try booking the class
    success = platform.book_class(user, test_class.class_id)
    
    # Print class details before and after booking
    print("\nTest Results:")
    print(f"Initial Capacity: 10")
    print(f"Booking Success: {success}")
    print(f"Remaining Capacity: {test_class.capacity}")
    print(f"Student Discount Applied: {user.is_student}")
    print(f"Final Price: {user.get_discounted_price(test_class.price):.2f} DKK")

# Run the test
test_pilates_platform()


Welcome Test User! You qualify for a 10% student discount!

=== Booking Confirmation ===
✓ Test User, your booking is confirmed!
Class: Test Pilates Class
Date: 2024-12-20 10:00
Price: 135.00 DKK (10% student discount applied)
Remaining Capacity: 9

Test Results:
Initial Capacity: 10
Booking Success: True
Remaining Capacity: 9
Student Discount Applied: True
Final Price: 135.00 DKK


## Example Usage

In [27]:
# Initialize the Pilates platform
platform = PilatesPlatform()

# Create sample areas with coordinates
areas = {
    "København Ø": (55.704, 12.579),
    "København K": (55.675, 12.568),
    "København N": (55.692, 12.543),
    "København V": (55.667, 12.541),
    "København S": (55.652, 12.590),
    "København NV": (55.700, 12.530)
}

# Create 20 classes (mix of different types)
for i in range(20):
    area_name, (lat, lng) = random.choice(list(areas.items()))
    class_type = random.choice([BeginnerPilatesClass, IntermediatePilatesClass, AdvancedPilatesClass])
    
    platform.add_class(class_type(
        name=f"Pilates Session {i+1}",
        date=generate_random_date(),
        duration=f"{random.randint(1, 2)}h",
        area=area_name,
        address=f"Studio {random.randint(1, 10)}, Copenhagen",
        instructor=random.choice(["Sarah", "Marie", "Philip", "Magnus"]),
        difficulty=random.choice(["Beginner", "Intermediate", "Advanced"]),
        price=generate_random_price(),
        latitude=lat + random.uniform(-0.01, 0.01),  # Add small random offset
        longitude=lng + random.uniform(-0.01, 0.01)
    ))

# Create 50 users (mix of students and regular users)
for i in range(50):
    is_student = random.choice([True, False])
    email = f"user{i}{'_student' if is_student else ''}@{'university.dk' if is_student else 'gmail.com'}"
    platform.register_user(f"User {i}", email)

# Create dataframes for visualization
df_classes = pd.DataFrame({
    'Class_ID': [cls.class_id for cls in platform.classes],
    'Name': [cls.name for cls in platform.classes],
    'Type': [cls.__class__.__name__ for cls in platform.classes],
    'Area': [cls.area for cls in platform.classes],
    'Price': [cls.price for cls in platform.classes],
    'Date': [cls.date for cls in platform.classes],
    'Latitude': [cls.latitude for cls in platform.classes],
    'Longitude': [cls.longitude for cls in platform.classes],
    'Capacity': [cls.capacity for cls in platform.classes]
})

df_users = pd.DataFrame({
    'User_ID': [user.user_id for user in platform.users],
    'Name': [user.name for user in platform.users],
    'Is_Student': [user.is_student for user in platform.users],
    'Email': [user.email for user in platform.users]
})

# Print summary statistics
print("\nPlatform Statistics:")
print(f"Total Classes: {len(platform.classes)}")
print(f"Total Users: {len(platform.users)}")
print(f"Student Users: {sum(1 for user in platform.users if user.is_student)}")
print("\nClasses by Type:")
print(df_classes['Type'].value_counts())
print("\nClasses by Area:")
print(df_classes['Area'].value_counts())
print(f"\nAverage Price: {df_classes['Price'].mean():.2f} DKK")

# Display the map
map_display = display_map_with_booking(platform, platform.users[0], sample_fraction=0.2)
display(map_display)


Welcome User 0! You qualify for a 10% student discount!

Welcome User 1! 

Welcome User 2! 

Welcome User 3! You qualify for a 10% student discount!

Welcome User 4! You qualify for a 10% student discount!

Welcome User 5! You qualify for a 10% student discount!

Welcome User 6! You qualify for a 10% student discount!

Welcome User 7! 

Welcome User 8! You qualify for a 10% student discount!

Welcome User 9! You qualify for a 10% student discount!

Welcome User 10! You qualify for a 10% student discount!

Welcome User 11! 

Welcome User 12! 

Welcome User 13! You qualify for a 10% student discount!

Welcome User 14! 

Welcome User 15! 

Welcome User 16! 

Welcome User 17! You qualify for a 10% student discount!

Welcome User 18! You qualify for a 10% student discount!

Welcome User 19! 

Welcome User 20! You qualify for a 10% student discount!

Welcome User 21! You qualify for a 10% student discount!

Welcome User 22! You qualify for a 10% student discount!

Welcome User 23! 

Welcome

In [29]:
df_users.head(5)

,User_ID,Name,Is_Student,Email
0,102,User 0,True,user0_student@university.dk
1,103,User 1,False,user1@gmail.com
2,104,User 2,False,user2@gmail.com
3,105,User 3,True,user3_student@university.dk
4,106,User 4,True,user4_student@university.dk


In [31]:
df_classes.head(5)

,Class_ID,Name,Type,Area,Price,Date,Latitude,Longitude,Capacity
0,52,Pilates Session 1,BeginnerPilatesClass,København S,119.03,2024-12-19 16:00,55.658514,12.583513,10
1,53,Pilates Session 2,BeginnerPilatesClass,København Ø,266.08,2024-12-25 7:00,55.713840,12.574240,10
2,54,Pilates Session 3,AdvancedPilatesClass,København N,235.26,2024-12-23 19:00,55.696586,12.548130,10
3,55,Pilates Session 4,BeginnerPilatesClass,København Ø,175.31,2024-12-25 15:00,55.699466,12.583294,10
4,56,Pilates Session 5,AdvancedPilatesClass,København N,287.47,2024-12-24 14:00,55.685319,12.534448,10


## Interactive Simulation

In [37]:
def interactive_simulation_with_map(platform):
    """Runs an interactive simulation with error handling."""
    print("\n------ Welcome to the Pilates Booking Platform ------")
    
    # User registration with validation
    while True:
        try:
            name = input("Please enter your name: ").strip()
            if not name:
                raise ValueError("Name cannot be empty")
            if any(char.isdigit() for char in name):
                raise ValueError("Name should not contain numbers")
                
            email = input("Please enter your email: ").strip().lower()
            if not email or '@' not in email or '.' not in email:
                raise ValueError("Please enter a valid email address")
                
            user = platform.register_user(name, email)
            break
        except ValueError as e:
            print(f"\nError: {e}")
            print("Please try again.\n")

    print(f"\nLet's find a Pilates class for you.")


    while True:
        try:
            print("\n--- Main Menu ---")
            print("1. Find Classes by Area")
            print("2. View Map of Classes")
            print("3. Exit")

            choice = input("\nEnter your choice: ").strip()
            if choice not in ["1", "2", "3"]:
                raise ValueError("Please enter 1, 2, or 3")

            if choice == "1":
                print("\n--- Choose Area to find classes---")
                areas = ["København Ø", "København K", "København N", 
                        "København V", "København S", "København NV"]
                for i, area in enumerate(areas, start=1):
                    print(f"{i}. {area}")
                
                try:
                    area_choice = int(input("\nSelect an area by number: ").strip())
                    if area_choice < 1 or area_choice > len(areas):
                        raise ValueError(f"Please enter a number between 1 and {len(areas)}")
                    chosen_area = areas[area_choice - 1]
                except ValueError as e:
                    print(f"\nError: Invalid input. {e}")
                    continue

                filtered_classes = platform.filter_classes_by_area(chosen_area)
                if not filtered_classes:
                    print("\nNo classes found in this area. Please try a different area.")
                    continue

                sorted_classes = platform.quicksort_classes(filtered_classes)
                print("\n--- Available Classes (Cheapest to Most Expensive) ---")
                for cls in sorted_classes:
                    print(cls.show_details())

                while True:
                    book_choice = input("\nWould you like to book a class? (y/n): ").strip().lower()
                    if book_choice in ['y', 'n']:
                        break
                    print("Please enter 'y' for yes or 'n' for no")

                if book_choice == "y":
                    try:
                        class_id = int(input("Enter the Class ID to book: ").strip())
                        if class_id <= 0:
                            raise ValueError("Class ID must be a positive number")
                        success = platform.book_class(user, class_id)
                        if not success:
                            print("\nPlease try another class or area.")
                    except ValueError as e:
                        print(f"\nError: {e}")

            elif choice == "2":
                try:
                    map_display = display_map_with_booking(platform, user, sample_fraction=0.2)
                    display(map_display)
                except Exception as e:
                    print(f"\nError displaying map: {e}")
                    print("Please try using the search option instead.")

            else:  # choice == "3"
                print("\nThank you for using the Pilates Booking Platform! Have a great day!")
                break

        except Exception as e:
            print(f"\nAn unexpected error occurred: {e}")
            print("Please try again.")

## Main Execution

In [39]:
def main():
    platform = PilatesPlatform()
    initialize_sample_data(platform)

    # Start the interactive simulation
    interactive_simulation_with_map(platform)

if __name__ == "__main__":
    main()


------ Welcome to the Pilates Booking Platform ------


Please enter your name:  Beata
Please enter your email:  bede@student.dk



Welcome Beata! You qualify for a 10% student discount!

Let's find a Pilates class for you.

--- Main Menu ---
1. Find Classes by Area
2. View Map of Classes
3. Exit



Enter your choice:  1



--- Choose Area to find classes---
1. København Ø
2. København K
3. København N
4. København V
5. København S
6. København NV


...


--- Available Classes (Cheapest to Most Expensive) ---
Class ID: 136, Name: Pilates Session 15, Date: 2024-12-20 19:00, Duration: 1h, Area: København Ø, Address: Studio 9, Copenhagen, Instructor: Sarah, Difficulty Level: Advanced, Price: 80.56 DKK, Remaining Capacity: 10
Class ID: 134, Name: Pilates Session 13, Date: 2024-12-23 10:00, Duration: 1h, Area: København Ø, Address: Studio 9, Copenhagen, Instructor: Philip, Difficulty Level: Advanced, Price: 84.65 DKK, Remaining Capacity: 10
Class ID: 133, Name: Pilates Session 12, Date: 2024-12-23 11:00, Duration: 1h, Area: København Ø, Address: Studio 2, Copenhagen, Instructor: Magnus, Difficulty Level: Advanced, Price: 183.41 DKK, Remaining Capacity: 10
Class ID: 141, Name: Pilates Session 20, Date: 2024-12-23 16:00, Duration: 1h, Area: København Ø, Address: Studio 4, Copenhagen, Instructor: Marie, Difficulty Level: Advanced, Price: 193.15 DKK, Remaining Capacity: 10
Class ID: 158, Name: Pilates Session 37, Date: 2024-12-19 17:00, Duratio


Would you like to book a class? (y/n):  n



--- Main Menu ---
1. Find Classes by Area
2. View Map of Classes
3. Exit



Enter your choice:  3



Thank you for using the Pilates Booking Platform! Have a great day!
